Import Games of Elo ~1100

In [3]:
import pandas as pd 
import torch 
from torch.utils.data import Dataset, DataLoader
import dask.dataframe as dd 

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Extract data from the dataframe at the given index
        # Format the data as required for your model
        data = self.dataframe.loc[idx].compute()
        data_tensor = torch.tensor(data, dtype=torch.float32)
        return data_tensor


In [40]:
# Import CSV File (from Maia: http://csslab.cs.toronto.edu/datasets/#monthly_chess_csv)
# The CSV has 151,072,060 rows
data_types ={'clock': 'float64',
       'cp': 'object',
       'opp_clock': 'float64'}
df = dd.read_csv('../data/lichess_db_standard_rated_2019-01.csv', blocksize='64e6', dtype= data_types)

# Select Relevant Columns
selected_columns = ['game_id','white_elo','black_elo','num_ply','move','white_active','board']
df = df[selected_columns]

# Filter out quick games (Bullet and HyperBullet)
condition_time_control = ~df['time_control'].isin(['Blitz', 'Bullet', 'HyperBullet'])
condition_clock = df['clock'] > 30
filtered_df = df[condition_time_control & condition_clock]

# Filter only games of Elo 1100-1199
filtered_df = filtered_df[(filtered_df['white_elo'].between(1100, 1199)) & (df['black_elo'].between(1100, 1199))]

# Group Same Games Together 
def aggregate_moves(group):
    moves = ' '.join(group['move'].compute())  # Concatenate moves into a single string
    white_elo = group['white_elo'].iloc[0].compute()  # Get the first white_elo
    black_elo = group['black_elo'].iloc[0].compute()  # Get the first black_elo
    num_ply = group['num_ply'].iloc[0].compute()  # Get the first num_ply
    return pd.Series({'moves': moves, 'white_elo': white_elo, 'black_elo': black_elo, 'num_ply': num_ply})

grouped_df = filtered_df.groupby('game_id').apply(aggregate_moves, meta={'moves': 'object', 'white_elo': 'int', 'black_elo': 'int', 'num_ply': 'int', 'white_active': 'str', 'board': 'str'}).compute()

# Samples 10K Rows
#first_10000_rows = df.head(1, compute =False)


print(df.compute())

# Generate DataLoader
#data = CustomDataset(df)
#dataloader = DataLoader(data, batch_size=64, shuffle=True)


/Users/jlee0/Desktop/cis400/enpoisson/.venv/lib/python3.9/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/jlee0/Desktop/cis400/enpoisson/.venv/lib/python3.9/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/jlee0/Desktop/cis400/enpoisson/.venv/lib/python3.9/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/jlee0/Desktop/cis400/enpoisson/.venv/lib/python3.9/site-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/jlee0/Desktop/cis400/enpoisson/.venv/lib/python3.9/site-packages/

         game_id  white_elo  black_elo  num_ply  move  white_active  \
1223    MpE9bxRV       1180       1125       72  e2e4          True   
1224    MpE9bxRV       1180       1125       72  e7e5         False   
1225    MpE9bxRV       1180       1125       72  g1f3          True   
1226    MpE9bxRV       1180       1125       72  f7f6         False   
1227    MpE9bxRV       1180       1125       72  f1b5          True   
...          ...        ...        ...      ...   ...           ...   
226501  L7OPs0AX       1160       1166       61  d6c5         False   
226502  L7OPs0AX       1160       1166       61  h5f5          True   
226503  L7OPs0AX       1160       1166       61  c8b8         False   
226504  L7OPs0AX       1160       1166       61  c3e4          True   
226506  L7OPs0AX       1160       1166       61  f5e6          True   

                                                    board  
1223    rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...  
1224    rnbqkbnr/pppppppp/8

Functions for turning raw data into sequential data

In [ ]:
import matplotlib.pyplot as plt
from datasets import load_dataset, concatenate_datasets
from torch.utils.data import DataLoader, Dataset
from utils import augment_with_noise
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from models import BiLSTMModel
from utils import get_glove_mapping
from nltk.tokenize import word_tokenize
from torch.optim.swa_utils import AveragedModel
import nltk
nltk.download('punkt')

class Vocabulary:
    def __init__(self):
        self.word_to_id = {"<PAD>": 0, "<UNK>": 1}
        self.id_to_word = {0: "<PAD>", 1: "<UNK>"}
        self.index = 2  # Start indexing from 2

    def add_word(self, word):
        if word not in self.word_to_id:
            self.word_to_id[word] = self.index
            self.id_to_word[self.index] = word
            self.index += 1

    def get_id(self, word):
        return self.word_to_id.get(word, self.word_to_id["<UNK>"])

# Function to convert sentences to sequences of indices
def text_to_sequence(texts, vocab_map):
    sequences = []
    for sentence in texts:
        seq = [vocab_map.get_id(word) for word in sentence]
        sequences.append(seq)
    return sequences

def get_embedding_matrix(vocab, n_embed, d_embed, glove_map, randomize_init = False):
    """
    Initialize the embedding matrix

    INPUT:
    n_embed         - size of the dictionary of embeddings
    d_embed         - the size of each embedding vector
    glove_map       - the map you created storing all of embeddings you will need from GloVE
    randomize_init  - if True, ignore the embeddings from glove_map and intilize all embeddings to random guassian noise (np.random.normal will be useful).

    OUTPUT:
    embedding_matrix  - a numpy matrix of mapping from word id to embedding

    """
    if randomize_init:
        return np.random.normal(0, 1, (n_embed, d_embed))
    embedding_matrix = np.zeros((n_embed, d_embed))
    for key in sorted(vocab.id_to_word.keys()):
        word = vocab.id_to_word[key]
        vector = glove_map[word] if word in glove_map else np.random.normal(0, 1, d_embed)
        embedding_matrix[key, np.arange(d_embed)] = vector
    return embedding_matrix

def pad_sequences(sequences, max_len=None, pad_id=0):
    if max_len is None:
        max_len = max(len(seq) for seq in sequences)
    padded_sequences = np.full((len(sequences), max_len), pad_id, dtype=int)
    sequence_lengths = np.zeros(len(sequences), dtype=int)
    for i, seq in enumerate(sequences):
        length = len(seq)
        padded_sequences[i, :length] = seq[:length]
        sequence_lengths[i] = length
    return torch.tensor(padded_sequences,dtype=torch.int64), torch.tensor(sequence_lengths,dtype=torch.int64)

class SequenceDataset(Dataset):
    def __init__(self, sequences, lengths, labels):
        self.sequences = sequences
        self.lengths = lengths
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.lengths[idx], self.labels[idx]

Functions for training

In [ ]:
# Import necessary libraries
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn as nn

In [1]:
# Bi-LSTM Model for PyTorch
class BiLSTMModel(nn.Module):
    def __init__(self, embedding_matrix, d_embed, d_hidden, d_out, dropout = 0.5, num_layers = 2, bidirectional = True):
        super(BiLSTMModel, self).__init__()
        self.embeddings = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(d_embed, d_hidden, dropout = dropout, bidirectional=bidirectional, num_layers = num_layers)
        self.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(2 * d_hidden * 2,d_out)
        )

    def forward(self, x, seq_lengths):
        x = self.embeddings(x)
        # Sort x and seq_lengths in descending order
        # This is required for packing the sequence
        seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
        x = x[perm_idx]
        # Pack the sequence
        packed_input = pack_padded_sequence(x, seq_lengths, batch_first=True)
        # Pass the packed sequence through the LSTM
        packed_output, (hidden, cell) = self.lstm(packed_input)

        # Unpack the sequence
        output, _ = pad_packed_sequence(packed_output, batch_first=True,total_length = x.size()[1])
        _, unperm_idx = perm_idx.sort(0)
        #unperm_idx = unperm_idx.to(self.device)
        output = output.index_select(0, unperm_idx)
        #This takes all the outputs across the cells
        mean_pooled = torch.mean(output, dim=1)
        max_pooled, _ = torch.max(output, dim=1)
        output = torch.cat((mean_pooled,max_pooled),dim=1)
        output = self.fc(output)
        return output

class MyDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]
    
# Training Function
def train_rnn(device, model, train_loader, val_loader, criterion, optimizer, num_epochs):
    train_loss_values = []
    train_error = []
    val_loss_values = []
    val_error = []
    swa_model = AveragedModel(model)
    swa_start = 1
    for epoch in range(num_epochs):
        train_correct = 0
        train_total = 0
        training_loss = 0.0
        # Training
        model.train()
        for sequences, lengths, labels in train_loader:
            sequences, lengths, labels = sequences.to(device), lengths.to(device), labels.to(device)
            # Forward Pass
            output = model(sequences, lengths)
            loss = criterion(output, labels)
            # Backpropogate & Optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # For logging purposes
            training_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        if epoch >= swa_start:
            swa_model.update_parameters(model)
        # Validation
        torch.optim.swa_utils.update_bn(train_loader, swa_model)

        model.eval()
        val_correct = 0
        val_total = 0
        validation_loss = 0.0
        if val_loader is not None:
            with torch.no_grad():
                for sequences, lengths, labels in val_loader:
                    sequences, lengths, labels = sequences.to(device), lengths.to(device), labels.to(device)
                    outputs = model(sequences, lengths)
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()
                    loss = criterion(outputs, labels)
                    validation_loss += loss.item()
            val_loss_values.append(validation_loss / len(val_loader))
            val_error.append(100-100*val_correct/val_total)
        # Log Model Performance  
        train_loss_values.append(training_loss)
        train_error.append(100-100*train_correct/train_total)
        print(f'Epoch {epoch+1}, Training Loss: {training_loss/len(train_loader)}, Validation Error: {val_error[-1]}, Training Error: {train_error[-1]}')
        for op_params in optimizer.param_groups:
            op_params['lr'] = op_params['lr'] * 0.6
    return train_error,train_loss_values, val_error, val_loss_values, swa_model

NameError: name 'Dataset' is not defined

Training Dataset

In [ ]:
# Import dataset
NUM_EPOCHS = 100
filepath = "/Users/jlee0/Desktop/cis400/enpoisson/lichess_db_standard_rated_2013-01.pgn"
raw_data = get_training_data_raw(3000)
X,Y = transform_data(raw_data)
data = MyDataset(X,Y)
train_loader = DataLoader(data, batch_size=64, shuffle=True)
raw_val_data = get_training_data_raw(500,validation=True)
X_val,Y_val = transform_data(raw_val_data)
val_data = MyDataset(X_val,Y_val)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True)

Processing Data into Sequences

In [ ]:
# Tokenize the sentences to get a vocabulary
def tokenize_dataset(dataset):
    return [word_tokenize(sentence['statement']) for sentence in dataset]
train_sequences = tokenize_dataset(train_dataset)
valid_sequences = tokenize_dataset(valid_dataset)
test_sequences = tokenize_dataset(test_dataset)

# Build vocab that handles going from token -> id
vocab = Vocabulary()
vocab_set = set()
for sentence in train_sequences:
    for word in sentence:
        vocab.add_word(word)
        vocab_set.add(word)

# Convert texts to sequences of indices using vocab
train_sequences = text_to_sequence(train_sequences, vocab)
valid_sequences = text_to_sequence(valid_sequences, vocab)
test_sequences = text_to_sequence(test_sequences, vocab)
# Glove Mapping that gives word -> embeddings 
glove_map = get_glove_mapping(vocab_set,"glove.840B.300d.txt")

d_out = 6  # the number of output classes of the model
n_embed = len(vocab.word_to_id) # the total number of word embeddings in the input layer
d_embed = 300 # the dimensionality of each word embedding

# Generate a word embeddings: id -> embedding using Glove & Vocab
embedding_matrix = torch.tensor(get_embedding_matrix(vocab, n_embed, d_embed, glove_map, randomize_init = False), dtype=torch.float32)

# Padding Sequence 
train_sequences, train_lengths = pad_sequences(train_sequences)
valid_sequences, valid_lengths = pad_sequences(valid_sequences)
test_sequences, test_lengths = pad_sequences(test_sequences)

# Preparing input
train_labels = torch.tensor(np.array([item['label'] for item in train_dataset]), dtype=torch.long)
valid_labels = torch.tensor(np.array([item['label'] for item in valid_dataset]), dtype=torch.long)
test_labels = torch.tensor(np.array([item['label'] for item in test_dataset]), dtype=torch.long)
# Create datasets and dataloaders
train_dataset = SequenceDataset(train_sequences, train_lengths, train_labels)
valid_dataset = SequenceDataset(valid_sequences, valid_lengths, valid_labels)
test_dataset = SequenceDataset(test_sequences, test_lengths, test_labels)
batch_size = 64 
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Training

In [ ]:
# Initialize model, loss function, and optimizer

#device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
d_hidden = 150
model = BiLSTMModel(embedding_matrix,d_embed,d_hidden,d_out,dropout=0.6,num_layers=2) 
model = model.to(device)
criterion = nn.CrossEntropyLoss()
lr = 5e-4
weight_decay=1e-4
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# Train the model
train_error,train_loss_values, val_error, val_loss_values = train(device, model, train_loader, val_loader, NUM_EPOCHS)

# Plot the training error
plt.figure(figsize=(10, 5))
plt.plot(val_error, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.title('Validation Error')
plt.legend()
plt.show()
plt.savefig('training_error_model_rnn.png')  # This will save the plot as an image